![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [1]:
import pandas as pd
import numpy as np
import logging
import os

# Start here...
logging.basicConfig(level=logging.INFO)

grocery_sales = pd.read_csv('./grocery_sales.csv')
grocery_sales['Date'] = pd.to_datetime(grocery_sales['Date'])
grocery_sales.info()

def extract(grocery_sales, parquet_file_path):
    extra_data_df = pd.read_parquet(parquet_file_path)
    merged_df = grocery_sales.merge(extra_data_df, on=['index'], how='inner')
    return merged_df

def transform(merged_df):
    merged_df.fillna({\
            'Weekly_Sales': merged_df['Weekly_Sales'].mean(),\
            'CPI': merged_df['CPI'].mean(),\
            'Unemployment': merged_df['Unemployment'].mean()\
        }, inplace = True)
    merged_df['Month'] = merged_df['Date'].dt.month
    merged_df = merged_df[merged_df['Weekly_Sales'] > 10000]
    clean_data = merged_df[['Store_ID', 'Month', 'Dept', 'IsHoliday', 'Weekly_Sales', 'CPI', 'Unemployment']]
    return clean_data

def avg_monthly_sales(clean_data):
    agg_data = clean_data.groupby('Month')['Weekly_Sales'].mean().reset_index()
    agg_data.columns = ['Month', 'Avg_Sales']
    agg_data['Avg_Sales'] = agg_data['Avg_Sales'].round(2)
    return agg_data

def load(clean_data, agg_data, clean_data_path, agg_data_path):
    clean_data.to_csv(clean_data_path, index=False)
    agg_data.to_csv(agg_data_path, index=False)

def validation(file_path):
    if os.path.exists(file_path):
        logging.info(f"Validation successful: {file_path} exist.")
    else:
        logging.error(f"Validation failed: {file_path} do not exist.")

merged_df = extract(grocery_sales, 'extra_data.parquet')
clean_data = transform(merged_df)
agg_data = avg_monthly_sales(clean_data)
load(clean_data, agg_data, 'clean_data.csv', 'agg_data.csv')
validation('clean_data.csv')
validation('agg_data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         20000 non-null  int64         
 1   Store_ID      20000 non-null  int64         
 2   Date          19961 non-null  datetime64[ns]
 3   Dept          20000 non-null  int64         
 4   Weekly_Sales  19962 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 781.4 KB


INFO:root:Validation successful: clean_data.csv exist.
INFO:root:Validation successful: agg_data.csv exist.
